In [22]:
import pickle
import nrrd
import numpy as np
from pathlib import Path
import os

In [33]:
def train_test_validation(volume):
    pct_80 = int(volume.shape[0] / 100 * 80)
    pct_90 = int((volume.shape[0] / 100 * 90))
    pct_100 = int(volume.shape[0])
    return volume[0:pct_80], volume[pct_80: pct_90], volume[pct_90:pct_100]

def write_pickle(obj, file):
    with open(file, 'b+w') as pout:
        pickle.dump(obj, pout)

def convert_to_tensor(seqs, segs):
    seqs = np.concatenate(seqs, axis=2)
    segs = np.concatenate(segs, axis=2)
    seqs = seqs.transpose((2, 0, 1))
    segs = segs.transpose((2, 0, 1))
    seqs, segs = np.expand_dims(seqs, axis=-1), np.expand_dims(segs, axis=-1)
    return seqs, segs

def get_files(sequences_path, label_path):
    sequence_paths, label_paths = [], []
    for seq, label in zip(sequences_path.glob("*.nrrd"), 
                        label_path.glob("*.nrrd")):
        sequence_paths.append(seq), label_paths.append(label)
    sequence_paths.sort(), label_paths.sort()
    return sequence_paths, label_paths

def crop_to_size(seqs, segs, x, y):
    return seqs[:,0:x,0:y,:], segs[:,0:x,0:y,:]

In [ ]:
names = ["T1", "T2", "T2star", "ASL", "DWI"]
base_dir = Path("/Users/jgray/Documents/Code/mri_kidney_perfusion/unet_training")
sequences = [Path(f"Dataset00{i}_{seq}") for (i, seq) in enumerate(names, start=1)]
labels = Path("labelsTr")
images = Path("imagesPr")


for i, seq_name in enumerate(names):

    output_path = base_dir/sequences[i]/Path("pickle")
    try:
        os.mkdir(output_path)
    except:
        print("folder exists")

    sequences_path = base_dir/sequences[i]/images
    label_path = base_dir/sequences[i]/labels

    seqs, labs = get_files(sequences_path, label_path)

    # convert to tensor
    seqs = [nrrd.read(path)[0] for path in seqs]
    labs = [nrrd.read(path)[0] for path in labs]

    seqs, segs = convert_to_tensor(seqs, labs)
    seqs, segs = crop_to_size(seqs, segs, 96, 128)

    seq_train, seq_test, seq_val = train_test_validation(seqs)
    seg_train, seg_test, seg_val = train_test_validation(segs)

    data = {"seg_test": seg_test,
            "seq_test": seq_test,
            "seg_train": seg_train,
            "seq_train": seq_train,
            "seg_val": seg_val,
            "seq_val": seq_val}

    write_pickle(data, output_path/f"{seq_name}.pkl")


folder exists
folder exists
folder exists
folder exists
folder exists


In [35]:
output_path = base_dir/sequences[-1]/Path("pickle")
try:
    os.mkdir(output_path)
except:
    print("folder exists")

sequences_path = base_dir/sequences[-1]/images
label_path = base_dir/sequences[-1]/labels

seqs, labs = get_files(sequences_path, label_path)

# convert to tensor
seqs = [nrrd.read(path)[0] for path in seqs]
labs = [nrrd.read(path)[0] for path in labs]

seqs, segs = convert_to_tensor(seqs, labs)
seqs, segs = crop_to_size(seqs, segs, 96, 128)

seq_train, seq_test, seq_val = train_test_validation(seqs)
seg_train, seg_test, seg_val = train_test_validation(segs)

data = {"seg_test": seg_test,
        "seq_test": seq_test,
        "seg_train": seg_train,
        "seq_train": seq_train,
        "seg_val": seg_val,
        "seq_val": seq_val}

write_pickle(data, output_path/f"{'DWI'}.pkl")

folder exists
